In [87]:
'''
import pandas as pd #引入Pandas模組
import csv

df=pd.read_csv('./file/ptt_dcard.csv',encoding='utf-8')
#encoding='windows-1252' -> 不加的話，會跑出下面的錯誤
#'utf-8' codec can't decode byte 0xa6 in position 0: invalid start byte

#1.刪掉不要的column
df.drop(labels=['resorce','title','author','url'],axis='columns',inplace=True)
#inplace=True 改變原本的檔案

#2.更換column的順序
df = df[['content', 'tag']]

#3.刪掉不要的tag 3 與 4
df=df[ df['tag'] < 3] #留下tag小於3的資料，因為我們要的tag是1與2
df.to_csv('for_csdn_1and2.csv',encoding='utf-8') #把結果存起來
'''

In [1]:
#df.head(10)

In [1]:
import json
import re
import jieba
import jieba.analyse
import numpy as np
import pandas as pd 
import nltk
import codecs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.cluster import KMeans

In [2]:

#加入stop word
with open(file='./mydict/stop_words.txt', mode='r', encoding='utf-8') as file:
    stop_words = file.read().split('\n')
    
stop_words.append('\n')  ## 文章中有許多分行符號，這邊加入停用字中，可以把它拿掉
stop_words.append('\n\n')

def jieba_tokenize(new_line):
    seg_stop_words_list = []
    seg_words_list = jieba.lcut(new_line)
    for term in seg_words_list:
        if term not in stop_words:
            seg_stop_words_list.append(term)
#seg_stop_words_list
    return seg_stop_words_list
    
    
    
    
tfidf_vectorizer = TfidfVectorizer(tokenizer=jieba_tokenize, lowercase=False)
'''
tokenizer: 指定分词函数
lowercase: 在分词之前将所有的文本转换成小写，因为涉及到中文文本处理，
所以最好是False
'''
#需要进行聚类的文本集
#text_list = ["今天天气真好啊啊啊啊", "我早上8点起床的", \
#"我今天拿到了Google的Offer", "清华大学在自然语言处理方面真厉害"]

#载入数据
#要使用絕對路徑,不然無法開啟
rows=pd.read_csv('C:/Users/leavi/Desktop/pyETL/file/final_for_svc.csv', header=0,encoding='utf-8',dtype=str)

text_list=[str(rows['content'])]
 
    
tfidf_matrix = tfidf_vectorizer.fit_transform(text_list)
 
k = 2

'''
n_clusters: 指定K的值
max_iter: 对于单次初始值计算的最大迭代次数
n_init: 重新选择初始值的次数
init: 制定初始值选择的算法
n_jobs: 进程个数，为-1的时候是指默认跑满CPU
注意，这个对于单个初始值的计算始终只会使用单进程计算，
并行计算只是针对与不同初始值的计算。比如n_init=10，n_jobs=40, 
服务器上面有20个CPU可以开40个进程，最终只会开10个进程
'''


for col in tfidf_matrix:
    kmeans = KMeans(n_clusters=k,max_iter=300, n_init=10,init='k-means++')
    X = tfidf_matrix.reshape(-1, 1)
    kmeans.fit(X)
    print("{}: {}".format(col, kmeans.predict(X)))


#返回各自文本的所被分配到的类索引
    #result = km_cluster.fit_predict(X)
 
    #print("Predicting result: ", result)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\leavi\AppData\Local\Temp\jieba.cache
Loading model cost 0.616 seconds.
Prefix dict has been built successfully.


  (0, 12)	0.08638684255813601
  (0, 9)	0.08638684255813601
  (0, 5)	0.08638684255813601
  (0, 6)	0.08638684255813601
  (0, 8)	0.08638684255813601
  (0, 7)	0.08638684255813601
  (0, 100)	0.08638684255813601
  (0, 64)	0.08638684255813601
  (0, 33)	0.08638684255813601
  (0, 75)	0.08638684255813601
  (0, 76)	0.08638684255813601
  (0, 112)	0.08638684255813601
  (0, 85)	0.08638684255813601
  (0, 36)	0.08638684255813601
  (0, 95)	0.08638684255813601
  (0, 38)	0.08638684255813601
  (0, 17)	0.08638684255813601
  (0, 4)	0.08638684255813601
  (0, 39)	0.08638684255813601
  (0, 25)	0.08638684255813601
  (0, 48)	0.08638684255813601
  (0, 82)	0.08638684255813601
  (0, 118)	0.17277368511627203
  (0, 20)	0.08638684255813601
  (0, 3)	0.08638684255813601
  :	:
  (0, 110)	0.08638684255813601
  (0, 42)	0.08638684255813601
  (0, 113)	0.08638684255813601
  (0, 94)	0.08638684255813601
  (0, 115)	0.08638684255813601
  (0, 79)	0.08638684255813601
  (0, 103)	0.25916052767440806
  (0, 19)	0.08638684255813601
  (0

In [5]:
import texthero as hero
import pandas as pd



df['tfidf'] = (
    rows['content']
    .pipe(hero.clean)
    .pipe(hero.tfidf(rows['content'],max_features=100))
)

df['kmeans_labels'] = (
    df['tfidf']
    .pipe(hero.kmeans, n_clusters=2)
    .astype(str)
)

df['pca'] = df['tfidf'].pipe(hero.pca)

hero.scatterplot(df, 'pca', color='kmeans_labels', title="K-means BBC Sport news")

ValueError: np.nan is an invalid document, expected byte or unicode string.